


# **Translational Data Augmentation** 

# **1. Uplaod the torch folder in MyDrive**

In [ ]:


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# **2. Clone or Upload DataAugmentationNMT folder in MyDrive**

In [ ]:
!git clone https://github.com/marziehf/DataAugmentationNMT




# **Step 1: Data Preprocessing**





**The input files train.en.cln , test.en.cln , dev.en.cln , vocab_freq.trg are uploaded in DataAugmentation folder**



**Before training the monolingual language model in [src/trg] you'll need to preprocess the data for both forward and backward direction using preprocess.no_preset_v.py.**

In [ ]:
%cd /content/gdrive/MyDrive/DataAugmentationNMT

/content/gdrive/MyDrive/DataAugmentationNMT


In [ ]:
!python src/preprocess.no_preset_v.py --train_txt /content/sample_data/train.en \
--val_txt /content/sample_data/dev.en --test_txt /content/sample_data/test.en \
--output_h5 ./data.h5 --output_json ./data.json

**Reverse the data for backward training**

In [ ]:
%cd /content/gdrive/MyDrive/DataAugmentationNMT


In [ ]:
!cat /content/gdrive/MyDrive/DataAugmentationNMT/train.en.te.cln | awk '{for (i=NF; i>0; i--) {printf "%s ", $i;} printf "\n" }' > train.en.rev.cln
!cat /content/gdrive/MyDrive/DataAugmentationNMT/test.en.te.cln | awk '{for (i=NF; i>0; i--) {printf "%s ", $i;} printf "\n" }' > test.en.rev.cln
!cat /content/gdrive/MyDrive/DataAugmentationNMT/dev.en.te.cln| awk '{for (i=NF; i>0; i--) {printf "%s ", $i;} printf "\n" }' > dev.en.rev.cln

In [ ]:
!python /content/gdrive/MyDrive/DataAugmentationNMT/src/preprocess.no_preset_v.py --train_txt ./train.en.rev.cln \
--val_txt ./dev.en.rev.cln --test_txt ./test.en.rev.cln \
--output_h5 ./data.rev.h5 --output_json ./data.rev.json

# **Step 2: Language Model Training in forward and backward direction** 


**Generation of vocabulary frequency list**

The vocabfreq input is the frequency list of words in the low-resource setting that need augmentation later on using these language models. The format is:

...

**change 3028**

**taken 3007**

**large 2999**

**again 2994**
...

***Script for Vocab Frequency file***

In [ ]:
import string 
 
# Open the file in read mode 
text = open("/content/gdrive/MyDrive/DataAugmentationNMT/ta/train.en.cln", "r") 
 
# Create an empty dictionary 
d = dict() 
 
# Loop through each line of the file 
for line in text: 
    # Remove the leading spaces and newline character 
    line = line.strip() 
 
    # Convert the characters in line to 
    # lowercase to avoid case mismatch 
    line = line.lower() 
 
    # Remove the punctuation marks from the line  
 
    # Split the line into words 
    words = line.split(" ") 
 
    # Iterate over each word in line 
    for word in words: 
        # Check if the word is already in dictionary 
        if word in d: 
            # Increment count of word by 1 
            d[word] = d[word] + 1
        else: 
            # Add the word to dictionary with count 1 
            d[word] = 1
import sys
from __future__ import print_function
f = open("vocab_freq.trg" , 'w')
sys.stdout =f
# Print the contents of dictionary 
for key in sorted(d, key=d.get, reverse=True): 
  sys.stdout =f
  print(key," ",d[key],sep="") 
 
f.close

<function TextIOWrapper.close>

**After preprocessing the data you'll need to train two language models in forward and backward directions.**

In [ ]:
!/content/gdrive/MyDrive/torch/install/bin/th /content/gdrive/MyDrive/torch/DataAugmentationNMT/src/train.lua -input_h5 data.h5 -input_json data.json \
-checkpoint_name /content/gdrive/MyDrive/DataAugmentationNMT/models_rnn/cv  -vocabfreq vocab_freq.trg -max_epochs 50 -print_every 2000 -gpu -1

In [ ]:
!/content/gdrive/MyDrive/torch/install/bin/th /content/gdrive/MyDrive/torch/DataAugmentationNMT/src/train.lua -input_h5 data.rev.h5 -input_json data.rev.json \
-checkpoint_name /content/gdrive/MyDrive/DataAugmentationNMT/models_rnn_rev/cv  -vocabfreq vocab_freq.trg -max_epochs 50 -print_every 2000 -gpu -1

# **Step 3: Substitution Generation**

**start_text** is the side of the bitext that you are targeting for augmentation of rare words. 

**vocabfreq** is the frequency list used for detecting rare words. 

**topk** indicates the maximum number of substitutions you want to have for each position in the sentence.

**After training the language models you can generate new sentences in your bitext for [src\trg]. You can run this:**

In [ ]:
!/content/gdrive/MyDrive/torch/install/bin/th ./src/substitution.lua -checkpoint /content/gdrive/MyDrive/DataAugmentationNMT/models_rnn/cv_100000.t7 -start_text /train.en.cln -vocabfreq /vocab_freq.ta.trg -sample 0 -topk 25 -bwd 0  > /subs.en -gpu -1

In [ ]:
!/content/gdrive/MyDrive/torch/install/bin/th ./src/substitution.lua -checkpoint /content/gdrive/MyDrive/DataAugmentationNMT/models_rnn_rev/cv_100000.t7 -start_text /train.en.rev.cln -vocabfreq /vocab_freq.trg -sample 0 -topk 25 -bwd 0  > /subs.en.rev -gpu -1

**Running these two codes will give you augmented corpora with a list of substitutions on one side subs.en and subs.en.rev In order to find substitions that best match the context, you'll need to find the intersection of these two lists:**

In [ ]:
!perl ./scripts/generate_intersect.pl /subs.en /subs.en.rev /subs.en.intersect

# **Step 4: Generate Augmented corpora**

Using the substitution output, the [trg/src] side of the bitext, the alignment, and the lexical probability file you can generate the augmented corpora.

You can use fast_align to obtain alignments for your bitext. The format of the alignment input is:


...

0-0 1-10 2-3 2-4 2-5 3-13 4-14 5-8 5-9 6-16 7-14 8-11 10-6 11-7 12-17

0-0 1-0 2-0 2-2 3-1 3-3 4-5 5-5 6-6 8-8 9-9 10-10 11-11

...



**fast_align** to obtain alignments for your bitext

In [ ]:
%cd /content/MyDrive

In [ ]:
! git clone https://github.com/clab/fast_align

In [ ]:
%cd /content/fast_align
%mkdir build
%cd /content/fast_align/build
!cmake ..
!make

 **Input format for fast align**

Input to fast_align must be tokenized and aligned into parallel sentences. Each line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space **(|||)**. An example 3-sentence German–English parallel corpus is:

**doch jetzt ist der Held gefallen . ||| but now the hero has fallen .**

**neue Modelle werden erprobt . ||| new models are being tested .**

**doch fehlen uns neue Ressourcen . ||| but we lack new resources**

In [ ]:
!./fast_align -i /content/fast_align/build/bilingual.txt -d -o -v  > /content/DataAugmentationNMT/alignment.txt

In [ ]:
!./fast_align -i /content/fast_align/build/bilingual.txt -d -o -v -p /content/DataAugmentationNMT/lex.prob

In [ ]:
%cd /content/DataAugmentationNMT

**To covert logrithimic lexical probability table into lexical probability table**

In [ ]:
import string 
import math
import sys
from __future__ import print_function
 
# Open the file in read mode 
text = open("lex.prob", "r") 

# Loop through each line of the file 
for line in text: 
    # Remove the leading spaces and newline character 
    line = line.strip()
    # Split the line into words 
    words = line.split()
    if len(words) >= 3:
      prob = words[2] 
      p= math.exp(float(prob))
      p=round(p,5)
      words= words[0] + " " + words[1] + " " + str(p)
      line = line.replace(line , words)
      with open('lex.txt', 'a') as f:
        print(line, file=f)


# **Augmented Corpa**

In [ ]:
!perl ./scripts/data_augmentation.pl /content/gdrive/MyDrive/DataAugmentationNMT/ta/subs.en.intersect /train.en.cln /alignment.txt /lex.txt /content/gdrive/MyDrive/DataAugmentationNMT/augmentedOutput.en

**This will generate two files: augmentedOutput.augmented in [src/trg] and augmentedOutput.fillout in [trg/src] language. The first file is the side of the bitext augmented targeting the rare words. The second file is respective translations of the augmented sentences.**

**If you want to have more than one change in each sentence you can also run:**

In [ ]:
!perl ./scripts/data_augmentation_multiplechanges.pl subs.intersect train.hi.cln alignment.txt lex.txt augmentedOutput

# **Step 5:Generate Clean Bitext for Translation**

Generate Clean Bitext for Translation
To remove all markups and have clean bitext that can be used for translation training you can run: 


In [ ]:
!perl ./scripts/filter_out_augmentations.pl augmentedOutput.en.augment.multmax.aug augmentedOutput.en.fillout.multmax.aug 1000


**You can impose further frequncy limit on rare words you want to augment here. The default frequency of rare word is 1000**